<a href="https://colab.research.google.com/github/reihaneh-torkzadehmahani/MyDPGAN/blob/master/AdvancedDPCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## differential_privacy.analysis.rdp_accountant

In [0]:
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""RDP analysis of the Sampled Gaussian Mechanism.

Functionality for computing Renyi differential privacy (RDP) of an additive
Sampled Gaussian Mechanism (SGM). Its public interface consists of two methods:
  compute_rdp(q, noise_multiplier, T, orders) computes RDP for SGM iterated
                                   T times.
  get_privacy_spent(orders, rdp, target_eps, target_delta) computes delta
                                   (or eps) given RDP at multiple orders and
                                   a target value for eps (or delta).

Example use:

Suppose that we have run an SGM applied to a function with l2-sensitivity 1.
Its parameters are given as a list of tuples (q1, sigma1, T1), ...,
(qk, sigma_k, Tk), and we wish to compute eps for a given delta.
The example code would be:

  max_order = 32
  orders = range(2, max_order + 1)
  rdp = np.zeros_like(orders, dtype=float)
  for q, sigma, T in parameters:
   rdp += rdp_accountant.compute_rdp(q, sigma, T, orders)
  eps, _, opt_order = rdp_accountant.get_privacy_spent(rdp, target_delta=delta)
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import sys

import numpy as np
from scipy import special
import six

########################
# LOG-SPACE ARITHMETIC #
########################


def _log_add(logx, logy):
    """Add two numbers in the log space."""
    a, b = min(logx, logy), max(logx, logy)
    if a == -np.inf:  # adding 0
        return b
    # Use exp(a) + exp(b) = (exp(a - b) + 1) * exp(b)
    return math.log1p(math.exp(a - b)) + b  # log1p(x) = log(x + 1)


def _log_sub(logx, logy):
    """Subtract two numbers in the log space. Answer must be non-negative."""
    if logx < logy:
        raise ValueError("The result of subtraction must be non-negative.")
    if logy == -np.inf:  # subtracting 0
        return logx
    if logx == logy:
        return -np.inf  # 0 is represented as -np.inf in the log space.

    try:
        # Use exp(x) - exp(y) = (exp(x - y) - 1) * exp(y).
        return math.log(
            math.expm1(logx - logy)) + logy  # expm1(x) = exp(x) - 1
    except OverflowError:
        return logx


def _log_print(logx):
    """Pretty print."""
    if logx < math.log(sys.float_info.max):
        return "{}".format(math.exp(logx))
    else:
        return "exp({})".format(logx)


def _compute_log_a_int(q, sigma, alpha):
    """Compute log(A_alpha) for integer alpha. 0 < q < 1."""
    assert isinstance(alpha, six.integer_types)

    # Initialize with 0 in the log space.
    log_a = -np.inf

    for i in range(alpha + 1):
        log_coef_i = (math.log(special.binom(alpha, i)) + i * math.log(q) +
                      (alpha - i) * math.log(1 - q))

        s = log_coef_i + (i * i - i) / (2 * (sigma**2))
        log_a = _log_add(log_a, s)

    return float(log_a)


def _compute_log_a_frac(q, sigma, alpha):
    """Compute log(A_alpha) for fractional alpha. 0 < q < 1."""
    # The two parts of A_alpha, integrals over (-inf,z0] and [z0, +inf), are
    # initialized to 0 in the log space:
    log_a0, log_a1 = -np.inf, -np.inf
    i = 0

    z0 = sigma**2 * math.log(1 / q - 1) + .5

    while True:  # do ... until loop
        coef = special.binom(alpha, i)
        log_coef = math.log(abs(coef))
        j = alpha - i

        log_t0 = log_coef + i * math.log(q) + j * math.log(1 - q)
        log_t1 = log_coef + j * math.log(q) + i * math.log(1 - q)

        log_e0 = math.log(.5) + _log_erfc((i - z0) / (math.sqrt(2) * sigma))
        log_e1 = math.log(.5) + _log_erfc((z0 - j) / (math.sqrt(2) * sigma))

        log_s0 = log_t0 + (i * i - i) / (2 * (sigma**2)) + log_e0
        log_s1 = log_t1 + (j * j - j) / (2 * (sigma**2)) + log_e1

        if coef > 0:
            log_a0 = _log_add(log_a0, log_s0)
            log_a1 = _log_add(log_a1, log_s1)
        else:
            log_a0 = _log_sub(log_a0, log_s0)
            log_a1 = _log_sub(log_a1, log_s1)

        i += 1
        if max(log_s0, log_s1) < -30:
            break

    return _log_add(log_a0, log_a1)


def _compute_log_a(q, sigma, alpha):
    """Compute log(A_alpha) for any positive finite alpha."""
    if float(alpha).is_integer():
        return _compute_log_a_int(q, sigma, int(alpha))
    else:
        return _compute_log_a_frac(q, sigma, alpha)


def _log_erfc(x):
    """Compute log(erfc(x)) with high accuracy for large x."""
    try:
        return math.log(2) + special.log_ndtr(-x * 2**.5)
    except NameError:
        # If log_ndtr is not available, approximate as follows:
        r = special.erfc(x)
        if r == 0.0:
            # Using the Laurent series at infinity for the tail of the erfc function:
            #     erfc(x) ~ exp(-x^2-.5/x^2+.625/x^4)/(x*pi^.5)
            # To verify in Mathematica:
            #     Series[Log[Erfc[x]] + Log[x] + Log[Pi]/2 + x^2, {x, Infinity, 6}]
            return (-math.log(math.pi) / 2 - math.log(x) - x**2 - .5 * x**-2 +
                    .625 * x**-4 - 37. / 24. * x**-6 + 353. / 64. * x**-8)
        else:
            return math.log(r)


def _compute_delta(orders, rdp, eps):
    """Compute delta given a list of RDP values and target epsilon.

  Args:
    orders: An array (or a scalar) of orders.
    rdp: A list (or a scalar) of RDP guarantees.
    eps: The target epsilon.

  Returns:
    Pair of (delta, optimal_order).

  Raises:
    ValueError: If input is malformed.

  """
    orders_vec = np.atleast_1d(orders)
    rdp_vec = np.atleast_1d(rdp)

    if len(orders_vec) != len(rdp_vec):
        raise ValueError("Input lists must have the same length.")

    deltas = np.exp((rdp_vec - eps) * (orders_vec - 1))
    idx_opt = np.argmin(deltas)
    return min(deltas[idx_opt], 1.), orders_vec[idx_opt]


def _compute_eps(orders, rdp, delta):
    """Compute epsilon given a list of RDP values and target delta.

  Args:
    orders: An array (or a scalar) of orders.
    rdp: A list (or a scalar) of RDP guarantees.
    delta: The target delta.

  Returns:
    Pair of (eps, optimal_order).

  Raises:
    ValueError: If input is malformed.

  """
    orders_vec = np.atleast_1d(orders)
    rdp_vec = np.atleast_1d(rdp)

    if len(orders_vec) != len(rdp_vec):
        raise ValueError("Input lists must have the same length.")

    eps = rdp_vec - math.log(delta) / (orders_vec - 1)

    idx_opt = np.nanargmin(eps)  # Ignore NaNs
    return eps[idx_opt], orders_vec[idx_opt]


def _compute_rdp(q, sigma, alpha):
    """Compute RDP of the Sampled Gaussian mechanism at order alpha.

  Args:
    q: The sampling rate.
    sigma: The std of the additive Gaussian noise.
    alpha: The order at which RDP is computed.

  Returns:
    RDP at alpha, can be np.inf.
  """
    if q == 0:
        return 0

    if q == 1.:
        return alpha / (2 * sigma**2)

    if np.isinf(alpha):
        return np.inf

    return _compute_log_a(q, sigma, alpha) / (alpha - 1)


def compute_rdp(q, noise_multiplier, steps, orders):
    """Compute RDP of the Sampled Gaussian Mechanism.

  Args:
    q: The sampling rate.
    noise_multiplier: The ratio of the standard deviation of the Gaussian noise
        to the l2-sensitivity of the function to which it is added.
    steps: The number of steps.
    orders: An array (or a scalar) of RDP orders.

  Returns:
    The RDPs at all orders, can be np.inf.
  """
    if np.isscalar(orders):
        rdp = _compute_rdp(q, noise_multiplier, orders)
    else:
        rdp = np.array(
            [_compute_rdp(q, noise_multiplier, order) for order in orders])

    return rdp * steps


def get_privacy_spent(orders, rdp, target_eps=None, target_delta=None):
    """Compute delta (or eps) for given eps (or delta) from RDP values.

  Args:
    orders: An array (or a scalar) of RDP orders.
    rdp: An array of RDP values. Must be of the same length as the orders list.
    target_eps: If not None, the epsilon for which we compute the corresponding
              delta.
    target_delta: If not None, the delta for which we compute the corresponding
              epsilon. Exactly one of target_eps and target_delta must be None.

  Returns:
    eps, delta, opt_order.

  Raises:
    ValueError: If target_eps and target_delta are messed up.
  """
    if target_eps is None and target_delta is None:
        raise ValueError(
            "Exactly one out of eps and delta must be None. (Both are).")

    if target_eps is not None and target_delta is not None:
        raise ValueError(
            "Exactly one out of eps and delta must be None. (None is).")

    if target_eps is not None:
        delta, opt_order = _compute_delta(orders, rdp, target_eps)
        return target_eps, delta, opt_order
    else:
        eps, opt_order = _compute_eps(orders, rdp, target_delta)
        return eps, target_delta, opt_order


## dp query



In [0]:
# Copyright 2018, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""An interface for differentially private query mechanisms.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc


class DPQuery(object):
  """Interface for differentially private query mechanisms."""

  __metaclass__ = abc.ABCMeta

  @abc.abstractmethod
  def initial_global_state(self):
    """Returns the initial global state for the DPQuery."""
    pass

  @abc.abstractmethod
  def derive_sample_params(self, global_state):
    """Given the global state, derives parameters to use for the next sample.

    Args:
      global_state: The current global state.

    Returns:
      Parameters to use to process records in the next sample.
    """
    pass

  @abc.abstractmethod
  def initial_sample_state(self, global_state, tensors):
    """Returns an initial state to use for the next sample.

    Args:
      global_state: The current global state.
      tensors: A structure of tensors used as a template to create the initial
        sample state.

    Returns: An initial sample state.
    """
    pass

  @abc.abstractmethod
  def accumulate_record(self, params, sample_state, record):
    """Accumulates a single record into the sample state.

    Args:
      params: The parameters for the sample.
      sample_state: The current sample state.
      record: The record to accumulate.

    Returns:
      The updated sample state.
    """
    pass

  @abc.abstractmethod
  def get_noised_result(self, sample_state, global_state):
    """Gets query result after all records of sample have been accumulated.

    Args:
      sample_state: The sample state after all records have been accumulated.
      global_state: The global state.

    Returns:
      A tuple (result, new_global_state) where "result" is the result of the
      query and "new_global_state" is the updated global state.
    """
    pass


## gausian query

In [0]:
# Copyright 2018, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Implements DPQuery interface for Gaussian average queries.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections

import tensorflow as tf

nest = tf.contrib.framework.nest


class GaussianSumQuery(DPQuery):
  """Implements DPQuery interface for Gaussian sum queries.

  Accumulates clipped vectors, then adds Gaussian noise to the sum.
  """

  # pylint: disable=invalid-name
  _GlobalState = collections.namedtuple(
      '_GlobalState', ['l2_norm_clip', 'stddev'])

  def __init__(self, l2_norm_clip, stddev):
    """Initializes the GaussianSumQuery.

    Args:
      l2_norm_clip: The clipping norm to apply to the global norm of each
        record.
      stddev: The stddev of the noise added to the sum.
    """
    self._l2_norm_clip = l2_norm_clip
    self._stddev = stddev

  def initial_global_state(self):
    """Returns the initial global state for the GaussianSumQuery."""
    return self._GlobalState(float(self._l2_norm_clip), float(self._stddev))

  def derive_sample_params(self, global_state):
    """Given the global state, derives parameters to use for the next sample.

    Args:
      global_state: The current global state.

    Returns:
      Parameters to use to process records in the next sample.
    """
    return global_state.l2_norm_clip

  def initial_sample_state(self, global_state, tensors):
    """Returns an initial state to use for the next sample.

    Args:
      global_state: The current global state.
      tensors: A structure of tensors used as a template to create the initial
        sample state.

    Returns: An initial sample state.
    """
    del global_state  # unused.
    return nest.map_structure(tf.zeros_like, tensors)

  def accumulate_record(self, params, sample_state, record):
    """Accumulates a single record into the sample state.

    Args:
      params: The parameters for the sample.
      sample_state: The current sample state.
      record: The record to accumulate.

    Returns:
      The updated sample state.
    """
    l2_norm_clip = params
    record_as_list = nest.flatten(record)
    clipped_as_list, _ = tf.clip_by_global_norm(record_as_list, l2_norm_clip)
    clipped = nest.pack_sequence_as(record, clipped_as_list)

    return nest.map_structure(tf.add, sample_state, clipped)

  def get_noised_result(self, sample_state, global_state, add_noise=True):
    """Gets noised sum after all records of sample have been accumulated.

    Args:
      sample_state: The sample state after all records have been accumulated.
      global_state: The global state.

    Returns:
      A tuple (estimate, new_global_state) where "estimate" is the estimated
      sum of the records and "new_global_state" is the updated global state.
    """
    def add_noise(v):
      if add_noise:
        return v + tf.random_normal(tf.shape(v), stddev=global_state.stddev)
      else:
        return v


    return nest.map_structure(add_noise, sample_state), global_state


class GaussianAverageQuery(DPQuery):
  """Implements DPQuery interface for Gaussian average queries.

  Accumulates clipped vectors, adds Gaussian noise, and normalizes.

  Note that we use "fixed-denominator" estimation: the denominator should be
  specified as the expected number of records per sample. Accumulating the
  denominator separately would also be possible but would be produce a higher
  variance estimator.
  """

  # pylint: disable=invalid-name
  _GlobalState = collections.namedtuple(
      '_GlobalState', ['sum_state', 'denominator'])

  def __init__(self, l2_norm_clip, sum_stddev, denominator):
    """Initializes the GaussianAverageQuery.

    Args:
      l2_norm_clip: The clipping norm to apply to the global norm of each
        record.
      sum_stddev: The stddev of the noise added to the sum (before
        normalization).
      denominator: The normalization constant (applied after noise is added to
        the sum).
    """
    self._numerator = GaussianSumQuery(l2_norm_clip, sum_stddev)
    self._denominator = denominator

  def initial_global_state(self):
    """Returns the initial global state for the GaussianAverageQuery."""
    sum_global_state = self._numerator.initial_global_state()
    return self._GlobalState(sum_global_state, float(self._denominator))

  def derive_sample_params(self, global_state):
    """Given the global state, derives parameters to use for the next sample.

    Args:
      global_state: The current global state.

    Returns:
      Parameters to use to process records in the next sample.
    """
    return self._numerator.derive_sample_params(global_state.sum_state)

  def initial_sample_state(self, global_state, tensors):
    """Returns an initial state to use for the next sample.

    Args:
      global_state: The current global state.
      tensors: A structure of tensors used as a template to create the initial
        sample state.

    Returns: An initial sample state.
    """
    # GaussianAverageQuery has no state beyond the sum state.
    return self._numerator.initial_sample_state(global_state.sum_state, tensors)

  def accumulate_record(self, params, sample_state, record):
    """Accumulates a single record into the sample state.

    Args:
      params: The parameters for the sample.
      sample_state: The current sample state.
      record: The record to accumulate.

    Returns:
      The updated sample state.
    """
    
    return self._numerator.accumulate_record(params, sample_state, record)

  def get_noised_result(self, sample_state, global_state, add_noise=True):
    """Gets noised average after all records of sample have been accumulated.

    Args:
      sample_state: The sample state after all records have been accumulated.
      global_state: The global state.

    Returns:
      A tuple (estimate, new_global_state) where "estimate" is the estimated
      average of the records and "new_global_state" is the updated global state.
    """
    noised_sum, new_sum_global_state = self._numerator.get_noised_result(
        sample_state, global_state.sum_state, add_noise)
    new_global_state = self._GlobalState(
        new_sum_global_state, global_state.denominator)
    def normalize(v):
      return tf.truediv(v, global_state.denominator)

    return nest.map_structure(normalize, noised_sum), new_global_state



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## our_dp_optimizer

In [0]:
# Copyright 2018, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Differentially private optimizers for TensorFlow."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


def make_optimizer_class(cls):
    """Constructs a DP optimizer class from an existing one."""
    if (tf.train.Optimizer.compute_gradients.__code__ is
            not cls.compute_gradients.__code__):
        tf.logging.warning(
            'WARNING: Calling make_optimizer_class() on class %s that overrides '
            'method compute_gradients(). Check to ensure that '
            'make_optimizer_class() does not interfere with overridden version.',
            cls.__name__)

    class DPOptimizerClass(cls):
        """Differentially private subclass of given class cls."""

        def __init__(
                self,
                l2_norm_clip,
                noise_multiplier,
                dp_average_query,
                num_microbatches,
                unroll_microbatches=False,
                *args,  # pylint: disable=keyword-arg-before-vararg
                **kwargs):
            super(DPOptimizerClass, self).__init__(*args, **kwargs)
            self._dp_average_query = dp_average_query
            self._num_microbatches = num_microbatches
            self._global_state = self._dp_average_query.initial_global_state()

            # TODO(b/122613513): Set unroll_microbatches=True to avoid this bug.
            # Beware: When num_microbatches is large (>100), enabling this parameter
            # may cause an OOM error.
            self._unroll_microbatches = unroll_microbatches

        def dp_compute_gradients(self,
                                 loss,
                                 var_list,
                                 gate_gradients=tf.train.Optimizer.GATE_OP,
                                 aggregation_method=None,
                                 colocate_gradients_with_ops=False,
                                 grad_loss=None,
                                 add_noise=True):

            # Note: it would be closer to the correct i.i.d. sampling of records if
            # we sampled each microbatch from the appropriate binomial distribution,
            # although that still wouldn't be quite correct because it would be
            # sampling from the dataset without replacement.
            microbatches_losses = tf.reshape(loss,
                                             [self._num_microbatches, -1])
            sample_params = (self._dp_average_query.derive_sample_params(
                self._global_state))

            def process_microbatch(i, sample_state):
                """Process one microbatch (record) with privacy helper."""
                grads, _ = zip(*super(cls, self).compute_gradients(
                    tf.gather(microbatches_losses, [i]), var_list,
                    gate_gradients, aggregation_method,
                    colocate_gradients_with_ops, grad_loss))

                # Converts tensor to list to replace None gradients with zero
                grads1 = list(grads)

                for inx in range(0, len(grads)):
                    if (grads[inx] == None):
                        grads1[inx] = tf.zeros_like(var_list[inx])

                grads_list = grads1
                sample_state = self._dp_average_query.accumulate_record(
                    sample_params, sample_state, grads_list)

                return sample_state

            if var_list is None:
                var_list = (tf.trainable_variables() + tf.get_collection(
                    tf.GraphKeys.TRAINABLE_RESOURCE_VARIABLES))
            sample_state = self._dp_average_query.initial_sample_state(
                self._global_state, var_list)

            if self._unroll_microbatches:
                for idx in range(self._num_microbatches):
                    sample_state = process_microbatch(idx, sample_state)
            else:
                # Use of while_loop here requires that sample_state be a nested
                # structure of tensors. In general, we would prefer to allow it to be
                # an arbitrary opaque type.

                cond_fn = lambda i, _: tf.less(i, self._num_microbatches)
                body_fn = lambda i, state: [
                    tf.add(i, 1), process_microbatch(i, state)
                ]

                idx = tf.constant(0)
                _, sample_state = tf.while_loop(cond_fn, body_fn,
                                                [idx, sample_state])

            final_grads, self._global_state = (
                self._dp_average_query.get_noised_result(
                    sample_state, self._global_state, add_noise))

            return (final_grads)

        def minimize(self,
                          d_loss_real,
                          d_loss_fake,
                          global_step=None,
                          var_list=None,
                          gate_gradients=tf.train.Optimizer.GATE_OP,
                          aggregation_method=None,
                          colocate_gradients_with_ops=False,
                          name=None,
                          grad_loss=None):
            """Minimize using sanitized gradients

            Args:
              d_loss_real: the loss tensor for real data
              d_loss_fake: the loss tensor for fake data
              global_step: the optional global step.
              var_list: the optional variables.
              name: the optional name.
            Returns:
              the operation that runs one step of DP gradient descent.
            """

            # First validate the var_list

            if var_list is None:
                var_list = tf.trainable_variables()
            for var in var_list:
                if not isinstance(var, tf.Variable):
                    raise TypeError("Argument is not a variable.Variable: %s" %
                                    var)

            # ------------------  OUR METHOD --------------------------------

            r_grads = self.dp_compute_gradients(
                d_loss_real,
                var_list=var_list,
                gate_gradients=gate_gradients,
                aggregation_method=aggregation_method,
                colocate_gradients_with_ops=colocate_gradients_with_ops,
                grad_loss=grad_loss, add_noise = True)

            f_grads = self.dp_compute_gradients(
                d_loss_fake,
                var_list=var_list,
                gate_gradients=gate_gradients,
                aggregation_method=aggregation_method,
                colocate_gradients_with_ops=colocate_gradients_with_ops,
                grad_loss=grad_loss,
                add_noise=False)

            # Compute the overall gradients 
            s_grads = [(r_grads[idx] + f_grads[idx])
                       for idx in range(len(r_grads))]

            sanitized_grads_and_vars = list(zip(s_grads, var_list))
            self._assert_valid_dtypes(
                [v for g, v in sanitized_grads_and_vars if g is not None])

            # Apply the overall gradients
            apply_grads = self.apply_gradients(sanitized_grads_and_vars,
                                               global_step=global_step,
                                               name=name)

            return apply_grads

         # -----------------------------------------------------------------

    return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
    """Constructs a DP optimizer with Gaussian averaging of updates."""

    class DPGaussianOptimizerClass(make_optimizer_class(cls)):
        """DP subclass of given class cls using Gaussian averaging."""

        def __init__(
                self,
                l2_norm_clip,
                noise_multiplier,
                num_microbatches,
                unroll_microbatches=False,
                *args,  # pylint: disable=keyword-arg-before-vararg
                **kwargs):
            dp_average_query = GaussianAverageQuery(
                l2_norm_clip, l2_norm_clip * noise_multiplier,
                num_microbatches)
            self.l2_norm_clip = l2_norm_clip
            self.noise_multiplier = noise_multiplier

            super(DPGaussianOptimizerClass,
                  self).__init__(l2_norm_clip, noise_multiplier,
                                 dp_average_query, num_microbatches,
                                 unroll_microbatches, *args, **kwargs)

    return DPGaussianOptimizerClass


DPAdagradOptimizer = make_optimizer_class(tf.train.AdagradOptimizer)
DPAdamOptimizer = make_optimizer_class(tf.train.AdamOptimizer)
DPGradientDescentOptimizer = make_optimizer_class(
    tf.train.GradientDescentOptimizer)

DPAdagradGaussianOptimizer = make_gaussian_optimizer_class(
    tf.train.AdagradOptimizer)
DPAdamGaussianOptimizer = make_gaussian_optimizer_class(tf.train.AdamOptimizer)
DPGradientDescentGaussianOptimizer = make_gaussian_optimizer_class(
    tf.train.GradientDescentOptimizer)


## gan.ops

In [0]:
"""
Most codes from https://github.com/carpedm20/DCGAN-tensorflow
"""
import math
import numpy as np
import tensorflow as tf


if "concat_v2" in dir(tf):

    def concat(tensors, axis, *args, **kwargs):
        return tf.concat_v2(tensors, axis, *args, **kwargs)
else:

    def concat(tensors, axis, *args, **kwargs):
        return tf.concat(tensors, axis, *args, **kwargs)


def bn(x, is_training, scope):
    return tf.contrib.layers.batch_norm(x,
                                        decay=0.9,
                                        updates_collections=None,
                                        epsilon=1e-5,
                                        scale=True,
                                        is_training=is_training,
                                        scope=scope)


def conv_out_size_same(size, stride):
    return int(math.ceil(float(size) / float(stride)))


def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.get_shape()
    y_shapes = y.get_shape()
    return concat(
        [x, y * tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])],
        3)


def conv2d(input_,
           output_dim,
           k_h=5,
           k_w=5,
           d_h=2,
           d_w=2,
           stddev=0.02,
           name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable(
            'w', [k_h, k_w, input_.get_shape()[-1], output_dim],
            initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_,
                            w,
                            strides=[1, d_h, d_w, 1],
                            padding='SAME')

        biases = tf.get_variable('biases', [output_dim],
                                 initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return conv


def deconv2d(input_,
             output_shape,
             k_h=5,
             k_w=5,
             d_h=2,
             d_w=2,
             name="deconv2d",
             stddev=0.02,
             with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable(
            'w', [k_h, k_w, output_shape[-1],
                  input_.get_shape()[-1]],
            initializer=tf.random_normal_initializer(stddev=stddev))

        try:
            deconv = tf.nn.conv2d_transpose(input_,
                                            w,
                                            output_shape=output_shape,
                                            strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_,
                                    w,
                                    output_shape=output_shape,
                                    strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]],
                                 initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv


def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak * x)


def linear(input_,
           output_size,
           scope=None,
           stddev=0.02,
           bias_start=0.0,
           with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
                               initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias


## OUR DP CGAN

In [0]:
# -*- coding: utf-8 -*-
from __future__ import division
from keras.datasets import cifar10

from mlxtend.data import loadlocal_mnist
from sklearn.preprocessing import label_binarize

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


class OUR_DP_CGAN(object):
    model_name = "OUR_DP_CGAN"  # name for checkpoint

    def __init__(self, sess, epoch, batch_size, z_dim, epsilon, delta, sigma,
                 clip_value, lr, dataset_name, base_dir, checkpoint_dir,
                 result_dir, log_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.base_dir = base_dir
        self.checkpoint_dir = checkpoint_dir
        self.result_dir = result_dir
        self.log_dir = log_dir
        self.epoch = epoch
        self.batch_size = batch_size
        self.epsilon = epsilon
        self.delta = delta
        self.noise_multiplier = sigma
        self.l2_norm_clip = clip_value
        self.lr = lr

        if dataset_name == 'mnist' or dataset_name == 'fashion-mnist':
            # parameters
            self.input_height = 28
            self.input_width = 28
            self.output_height = 28
            self.output_width = 28

            self.z_dim = z_dim  # dimension of noise-vector
            self.y_dim = 10  # dimension of condition-vector (label)
            self.c_dim = 1

            # train
            self.learningRateD = self.lr
            self.learningRateG = self.learningRateD * 5
            self.beta1 = 0.5
            self.beta2 = 0.99
            # test
            self.sample_num = 64  # number of generated images to be saved

            # load mnist
            self.data_X, self.data_y = load_mnist(train = True)

            # get number of batches for a single epoch
            self.num_batches = len(self.data_X) // self.batch_size

        elif dataset_name == 'cifar10':
            # parameters
            self.input_height = 32
            self.input_width = 32
            self.output_height = 32
            self.output_width = 32

            self.z_dim = 100  # dimension of noise-vector
            self.y_dim = 10  # dimension of condition-vector (label)
            self.c_dim = 3  # color dimension

            # train
            # self.learning_rate = 0.0002 # 1e-3, 1e-4
            self.learningRateD = 1e-3
            self.learningRateG = 1e-4
            self.beta1 = 0.5
            self.beta2 = 0.99

            # test
            self.sample_num = 64  # number of generated images to be saved

            # load cifar10
            self.data_X, self.data_y = load_cifar10(train=True)

            self.num_batches = len(self.data_X) // self.batch_size

        else:
            raise NotImplementedError

    def discriminator(self, x, y, is_training=True, reuse=False):
        # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
        # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
        with tf.variable_scope("discriminator", reuse=reuse):

            # merge image and label
            if (self.dataset_name == "mnist"):
                y = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
                x = conv_cond_concat(x, y)

                net = lrelu(conv2d(x, 64, 4, 4, 2, 2, name='d_conv1'))
                net = lrelu(
                    bn(conv2d(net, 128, 4, 4, 2, 2, name='d_conv2'),
                       is_training=is_training,
                       scope='d_bn2'))
                net = tf.reshape(net, [self.batch_size, -1])
                net = lrelu(
                    bn(linear(net, 1024, scope='d_fc3'),
                       is_training=is_training,
                       scope='d_bn3'))
                out_logit = linear(net, 1, scope='d_fc4')
                out = tf.nn.sigmoid(out_logit)

            elif (self.dataset_name == "cifar10"):

                y = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
                x = conv_cond_concat(x, y)
                lrelu_slope = 0.2
                kernel_size = 5
                w_init = tf.contrib.layers.xavier_initializer()

                net = lrelu(
                    conv2d(x,
                           64,
                           5,
                           5,
                           2,
                           2,
                           name='d_conv1' + '_' + self.dataset_name))
               
                net = lrelu(
                    bn(conv2d(net,
                              128,
                              5,
                              5,
                              2,
                              2,
                              name='d_conv2' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='d_bn2'))
         
                net = lrelu(
                    bn(conv2d(net,
                              256,
                              5,
                              5,
                              2,
                              2,
                              name='d_conv3' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='d_bn3'))
                
                net = lrelu(
                    bn(conv2d(net,
                              512,
                              5,
                              5,
                              2,
                              2,
                              name='d_conv4' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='d_bn4'))
               
                net = tf.reshape(net, [self.batch_size, -1])
               
                out_logit = linear(net,
                                   1,
                                   scope='d_fc5' + '_' + self.dataset_name)
                
                out = tf.nn.sigmoid(out_logit)

            return out, out_logit

    def generator(self, z, y, is_training=True, reuse=False):
        # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
        # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
        with tf.variable_scope("generator", reuse=reuse):

            if (self.dataset_name == "mnist"):
                # merge noise and label
                z = concat([z, y], 1)

                net = tf.nn.relu(
                    bn(linear(z, 1024, scope='g_fc1'),
                       is_training=is_training,
                       scope='g_bn1'))
                net = tf.nn.relu(
                    bn(linear(net, 128 * 7 * 7, scope='g_fc2'),
                       is_training=is_training,
                       scope='g_bn2'))
                net = tf.reshape(net, [self.batch_size, 7, 7, 128])
                net = tf.nn.relu(
                    bn(deconv2d(net, [self.batch_size, 14, 14, 64],
                                4,
                                4,
                                2,
                                2,
                                name='g_dc3'),
                       is_training=is_training,
                       scope='g_bn3'))

                out = tf.nn.sigmoid(
                    deconv2d(net, [self.batch_size, 28, 28, 1],
                             4,
                             4,
                             2,
                             2,
                             name='g_dc4'))

            elif (self.dataset_name == "cifar10"):
                h_size = 32
                h_size_2 = 16
                h_size_4 = 8
                h_size_8 = 4
                h_size_16 = 2

                z = concat([z, y], 1)

                net = linear(z,
                             512 * h_size_16 * h_size_16,
                             scope='g_fc1' + '_' + self.dataset_name)
                
                net = tf.nn.relu(
                    bn(tf.reshape(
                        net, [self.batch_size, h_size_16, h_size_16, 512]),
                       is_training=is_training,
                       scope='g_bn1'))
                
                net = tf.nn.relu(
                    bn(deconv2d(net,
                                [self.batch_size, h_size_8, h_size_8, 256],
                                5,
                                5,
                                2,
                                2,
                                name='g_dc2' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='g_bn2'))
               
                net = tf.nn.relu(
                    bn(deconv2d(net,
                                [self.batch_size, h_size_4, h_size_4, 128],
                                5,
                                5,
                                2,
                                2,
                                name='g_dc3' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='g_bn3'))
                
                net = tf.nn.relu(
                    bn(deconv2d(net, [self.batch_size, h_size_2, h_size_2, 64],
                                5,
                                5,
                                2,
                                2,
                                name='g_dc4' + '_' + self.dataset_name),
                       is_training=is_training,
                       scope='g_bn4'))
                
                out = tf.nn.tanh(
                    deconv2d(net, [
                        self.batch_size, self.output_height, self.output_width,
                        self.c_dim
                    ],
                             5,
                             5,
                             2,
                             2,
                             name='g_dc5' + '_' + self.dataset_name))

            return out

    def build_model(self):
        # some parameters
        image_dims = [self.input_height, self.input_width, self.c_dim]
        bs = self.batch_size
        
        """ Graph Input """
        # images
        self.inputs = tf.placeholder(tf.float32, [bs] + image_dims,
                                     name='real_images')

        # labels
        self.y = tf.placeholder(tf.float32, [bs, self.y_dim], name='y')

        # noises
        self.z = tf.placeholder(tf.float32, [bs, self.z_dim], name='z')
        """ Loss Function """

        # output of D for real images
        D_real, D_real_logits = self.discriminator(self.inputs,
                                                   self.y,
                                                   is_training=True,
                                                   reuse=False)

        # output of D for fake images
        G = self.generator(self.z, self.y, is_training=True, reuse=False)
        D_fake, D_fake_logits = self.discriminator(G,
                                                   self.y,
                                                   is_training=True,
                                                   reuse=True)

        # get loss for discriminator
        d_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_real_logits, labels=tf.ones_like(D_real)))
        d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_fake_logits, labels=tf.zeros_like(D_fake)))

        self.d_loss_real_vec = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=D_real_logits, labels=tf.ones_like(D_real))
        self.d_loss_fake_vec = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=D_fake_logits, labels=tf.zeros_like(D_fake))

        self.d_loss = d_loss_real + d_loss_fake

        # get loss for generator
        self.g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_fake_logits, labels=tf.ones_like(D_fake)))
        
        """ Training """
        # divide trainable variables into a group for D and a group for G
        t_vars = tf.trainable_variables()
        d_vars = [
            var for var in t_vars if var.name.startswith('discriminator')
        ]
        g_vars = [var for var in t_vars if var.name.startswith('generator')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(
                tf.GraphKeys.UPDATE_OPS)):

            d_optim_init = DPGradientDescentGaussianOptimizer(
                l2_norm_clip=self.l2_norm_clip,
                noise_multiplier=self.noise_multiplier,
                num_microbatches=self.batch_size,
                learning_rate=self.learningRateD)

            global_step = tf.train.get_global_step()

            self.d_optim = d_optim_init.minimize(
                d_loss_real=self.d_loss_real_vec,
                d_loss_fake=self.d_loss_fake_vec,
                global_step=global_step,
                var_list=d_vars)

            optimizer = DPGradientDescentGaussianOptimizer(
                l2_norm_clip=self.l2_norm_clip,
                noise_multiplier=self.noise_multiplier,
                num_microbatches=self.batch_size,
                learning_rate=self.learningRateD)

            self.g_optim = tf.train.GradientDescentOptimizer(self.learningRateG) \
                        .minimize(self.g_loss, var_list=g_vars)
            
        """" Testing """
        self.fake_images = self.generator(self.z,
                                          self.y,
                                          is_training=False,
                                          reuse=True)
        """ Summary """
        d_loss_real_sum = tf.summary.scalar("d_loss_real", d_loss_real)
        d_loss_fake_sum = tf.summary.scalar("d_loss_fake", d_loss_fake)
        d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)

        # final summary operations
        self.g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
        self.d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])

    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # graph inputs for visualize training results
        self.sample_z = np.random.uniform(-1,
                                          1,
                                          size=(self.batch_size, self.z_dim))
        self.test_labels = self.data_y[0:self.batch_size]

        # saver to save model
        self.saver = tf.train.Saver()

        # summary writer
        self.writer = tf.summary.FileWriter(
            self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_counter = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_counter / self.num_batches)
            start_batch_id = checkpoint_counter - start_epoch * self.num_batches
            counter = checkpoint_counter
            print(" [*] Load SUCCESS")
        else:
            start_epoch = 0
            start_batch_id = 0
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        epoch = start_epoch
        should_terminate = False
        
        while (epoch < self.epoch and not should_terminate):

            # get batch data
            for idx in range(start_batch_id, self.num_batches):
                batch_images = self.data_X[idx * self.batch_size:(idx + 1) *
                                           self.batch_size]
                batch_labels = self.data_y[idx * self.batch_size:(idx + 1) *
                                           self.batch_size]
                batch_z = np.random.uniform(
                    -1, 1, [self.batch_size, self.z_dim]).astype(np.float32)

                # update D network

                _, summary_str, d_loss = self.sess.run(
                    [self.d_optim, self.d_sum, self.d_loss],
                    feed_dict={
                        self.inputs: batch_images,
                        self.y: batch_labels,
                        self.z: batch_z
                    })
                self.writer.add_summary(summary_str, counter)

                eps = self.compute_epsilon((epoch * self.num_batches) + idx)

                if (eps > self.epsilon):

                    should_terminate = True
                    print("TERMINATE !! Run out of Privacy Budget.....")
                    epoch = self.epoch
                    break

                # update G network
                _, summary_str, g_loss = self.sess.run(
                    [self.g_optim, self.g_sum, self.g_loss],
                    feed_dict={
                        self.inputs: batch_images,
                        self.y: batch_labels,
                        self.z: batch_z
                    })

                self.writer.add_summary(summary_str, counter)

                # display training status
                counter += 1
                _ = self.sess.run(self.fake_images,
                                  feed_dict={
                                      self.z: self.sample_z,
                                      self.y: self.test_labels
                                  })

                # save training results for every 100 steps
                if np.mod(counter, 100) == 0:
                    print("Iteration : " + str(idx) + " Eps: " + str(eps))
                    samples = self.sess.run(self.fake_images,
                                            feed_dict={
                                                self.z: self.sample_z,
                                                self.y: self.test_labels
                                            })
                    tot_num_samples = min(self.sample_num, self.batch_size)
                    manifold_h = int(np.floor(np.sqrt(tot_num_samples)))
                    manifold_w = int(np.floor(np.sqrt(tot_num_samples)))
                    save_images(
                        samples[:manifold_h * manifold_w, :, :, :],
                        [manifold_h, manifold_w],
                        check_folder(self.result_dir + '/' + self.model_dir) +
                        '/' + self.model_name +
                        '_train_{:02d}_{:04d}.png'.format(epoch, idx))
            epoch = epoch + 1

            # After an epoch, start_batch_id is set to zero
            # non-zero value is only for the first epoch after loading pre-trained model
            start_batch_id = 0

            # save model
            self.save(self.checkpoint_dir, counter)

            # show temporal results
            if (self.dataset_name == 'mnist'):
                self.visualize_results_MNIST(epoch)
            elif (self.dataset_name == 'cifar10'):
                self.visualize_results_CIFAR(epoch)

        # save model for final step
        self.save(self.checkpoint_dir, counter)


        def compute_fpr_tpr_roc(Y_test, Y_score):
            n_classes = Y_score.shape[1]
            false_positive_rate = dict()
            true_positive_rate = dict()
            roc_auc = dict()
            for class_cntr in range(n_classes):
                false_positive_rate[class_cntr], true_positive_rate[
                    class_cntr], _ = roc_curve(Y_test[:, class_cntr],
                                               Y_score[:, class_cntr])
                roc_auc[class_cntr] = auc(false_positive_rate[class_cntr],
                                          true_positive_rate[class_cntr])

            # Compute micro-average ROC curve and ROC area
            false_positive_rate["micro"], true_positive_rate[
                "micro"], _ = roc_curve(Y_test.ravel(), Y_score.ravel())
            roc_auc["micro"] = auc(false_positive_rate["micro"],
                                   true_positive_rate["micro"])

            return false_positive_rate, true_positive_rate, roc_auc

        def classify(X_train,
                     Y_train,
                     X_test,
                     classiferName,
                     random_state_value=0):
            if classiferName == "lr":
                classifier = OneVsRestClassifier(
                    LogisticRegression(solver='lbfgs',
                                       multi_class='multinomial',
                                       random_state=random_state_value))
            elif classiferName == "mlp":
                classifier = OneVsRestClassifier(
                    MLPClassifier(random_state=random_state_value, alpha=1))

            elif classiferName == "rf":
                classifier = OneVsRestClassifier(
                    RandomForestClassifier(n_estimators=100,
                                           random_state=random_state_value))

            else:
                print("Classifier not in the list!")
                exit()
            Y_score = classifier.fit(X_train, Y_train).predict_proba(X_test)
            return Y_score

        batch_size = int(self.batch_size)

        if (self.dataset_name == "mnist"):

            n_class = np.zeros(10)
            n_class[0] = 5923 - batch_size
            n_class[1] = 6742
            n_class[2] = 5958
            n_class[3] = 6131
            n_class[4] = 5842
            n_class[5] = 5421
            n_class[6] = 5918
            n_class[7] = 6265
            n_class[8] = 5851
            n_class[9] = 5949

            Z_sample = np.random.uniform(-1, 1, size=(batch_size, self.z_dim))
            y = np.zeros(batch_size, dtype=np.int64) + 0
            y_one_hot = np.zeros((batch_size, self.y_dim))
            y_one_hot[np.arange(batch_size), y] = 1
            images = self.sess.run(self.fake_images,
                                   feed_dict={
                                       self.z: Z_sample,
                                       self.y: y_one_hot
                                   })

            for classLabel in range(0, 10):
                for _ in range(0, int(n_class[classLabel]), batch_size):
                    Z_sample = np.random.uniform(-1,
                                                 1,
                                                 size=(batch_size, self.z_dim))
                    y = np.zeros(batch_size, dtype=np.int64) + classLabel
                    y_one_hot_init = np.zeros((batch_size, self.y_dim))
                    y_one_hot_init[np.arange(batch_size), y] = 1

                    images = np.append(images,
                                       self.sess.run(self.fake_images,
                                                     feed_dict={
                                                         self.z: Z_sample,
                                                         self.y: y_one_hot_init
                                                     }),
                                       axis=0)
                    y_one_hot = np.append(y_one_hot, y_one_hot_init, axis=0)

            X_test, Y_test = load_mnist(train = False)

            Y_test = [int(y) for y in Y_test]

            classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
            Y_test = label_binarize(Y_test, classes=classes)

        if (self.dataset_name == "cifar10"):
            n_class = np.zeros(10)
            for t in range(1, 10):
                n_class[t] = 1000
            
            Z_sample = np.random.uniform(-1, 1, size=(batch_size, self.z_dim))
            y = np.zeros(batch_size, dtype=np.int64) + 0
            y_one_hot = np.zeros((batch_size, self.y_dim))
            y_one_hot[np.arange(batch_size), y] = 1
            images = self.sess.run(self.fake_images,
                                   feed_dict={
                                       self.z: Z_sample,
                                       self.y: y_one_hot
                                   })

            for classLabel in range(0, 10):
                for _ in range(0, int(n_class[classLabel]), batch_size):
                    Z_sample = np.random.uniform(-1,
                                                 1,
                                                 size=(batch_size, self.z_dim))
                    y = np.zeros(batch_size, dtype=np.int64) + classLabel
                    y_one_hot_init = np.zeros((batch_size, self.y_dim))
                    y_one_hot_init[np.arange(batch_size), y] = 1

                    images = np.append(images,
                                       self.sess.run(self.fake_images,
                                                     feed_dict={
                                                         self.z: Z_sample,
                                                         self.y: y_one_hot_init
                                                     }),
                                       axis=0)
                    y_one_hot = np.append(y_one_hot, y_one_hot_init, axis=0)

            X_test, Y_test = load_cifar10(train=False)

            classes = range(0, 10)
            Y_test = label_binarize(Y_test, classes=classes)

        print("  Classifying - Logistic Regression...")

        TwoDim_images = images.reshape(np.shape(images)[0], -2)
 
        X_test = X_test.reshape(np.shape(X_test)[0], -2)
        Y_score = classify(TwoDim_images,
                           y_one_hot,
                           X_test,
                           "lr",
                           random_state_value=30)

        false_positive_rate, true_positive_rate, roc_auc = compute_fpr_tpr_roc(
            Y_test, Y_score)

        classification_results_fname = self.base_dir + "CGAN_AuROC.txt"
        classification_results = open(classification_results_fname, "w")

        classification_results.write(
            "\nepsilon : {:.2f}, sigma: {:.2f}, clipping value: {:.2f}".format(
                (self.epsilon), round(self.noise_multiplier, 2),
                round(self.l2_norm_clip, 2)))

        classification_results.write("\nAuROC - logistic Regression: " +
                                     str(roc_auc["micro"]))
        classification_results.write(
            "\n--------------------------------------------------------------------\n"
        )
      
        print("  Classifying - Random Forest...")
        Y_score = classify(TwoDim_images,
                           y_one_hot,
                           X_test,
                           "rf",
                           random_state_value=30)

        print("  Computing ROC - Random Forest ...")
        false_positive_rate, true_positive_rate, roc_auc = compute_fpr_tpr_roc(
            Y_test, Y_score)

        classification_results.write(
            "\nepsilon : {:.2f}, sigma: {:.2f}, clipping value: {:.2f}".format(
                (self.epsilon), round(self.noise_multiplier, 2),
                round(self.l2_norm_clip, 2)))

        classification_results.write("\nAuROC - random Forest: " +
                                     str(roc_auc["micro"]))
        classification_results.write(
            "\n--------------------------------------------------------------------\n"
        )
      
        print("  Classifying - multilayer Perceptron ...")
        Y_score = classify(TwoDim_images,
                           y_one_hot,
                           X_test,
                           "mlp",
                           random_state_value=30)

        print("  Computing ROC - Multilayer Perceptron ...")
        false_positive_rate, true_positive_rate, roc_auc = compute_fpr_tpr_roc(
            Y_test, Y_score)

        classification_results.write(
            "\nepsilon : {:.2f}, sigma: {:.2f}, clipping value: {:.2f}".format(
                (self.epsilon), round(self.noise_multiplier, 2),
                round(self.l2_norm_clip, 2)))

        classification_results.write("\nAuROC - multilayer Perceptron: " +
                                     str(roc_auc["micro"]))
        classification_results.write(
            "\n--------------------------------------------------------------------\n"
        )

        # save model for final step
        self.save(self.checkpoint_dir, counter)

    def compute_epsilon(self, steps):
        """Computes epsilon value for given hyperparameters."""
        if self.noise_multiplier == 0.0:
            return float('inf')
        orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
        sampling_probability = self.batch_size / 60000
        rdp = compute_rdp(q=sampling_probability,
                          noise_multiplier=self.noise_multiplier,
                          steps=steps,
                          orders=orders)
        # Delta is set to 1e-5 because MNIST has 60000 training points.
        return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

    # CIFAR 10
    def visualize_results_CIFAR(self, epoch):
        tot_num_samples = min(self.sample_num, self.batch_size)  # 64, 100
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples)))  # 8
        """ random condition, random noise """
        y = np.random.choice(self.y_dim, self.batch_size)
        y_one_hot = np.zeros((self.batch_size, self.y_dim))
        y_one_hot[np.arange(self.batch_size), y] = 1

        z_sample = np.random.uniform(-1, 1, size=(self.batch_size,
                                                  self.z_dim))  # 100, 100

        samples = self.sess.run(self.fake_images,
                                feed_dict={
                                    self.z: z_sample,
                                    self.y: y_one_hot
                                })

        save_matplot_img(
            samples[:image_frame_dim * image_frame_dim, :, :, :],
            [image_frame_dim, image_frame_dim], self.result_dir + '/' +
            self.model_name + '_epoch%03d' % epoch + '_test_all_classes.png')

    # MNIST
    def visualize_results_MNIST(self, epoch):
        tot_num_samples = min(self.sample_num, self.batch_size)
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples)))
        """ random condition, random noise """
        y = np.random.choice(self.y_dim, self.batch_size)
        y_one_hot = np.zeros((self.batch_size, self.y_dim))
        y_one_hot[np.arange(self.batch_size), y] = 1

        z_sample = np.random.uniform(-1, 1, size=(self.batch_size, self.z_dim))
        samples = self.sess.run(self.fake_images,
                                feed_dict={
                                    self.z: z_sample,
                                    self.y: y_one_hot
                                })

        save_images(
            samples[:image_frame_dim * image_frame_dim, :, :, :],
            [image_frame_dim, image_frame_dim],
            check_folder(self.result_dir + '/' + self.model_dir) + '/' +
            self.model_name + '_epoch%03d' % epoch + '_test_all_classes.png')
        """ specified condition, random noise """
        n_styles = 10  # must be less than or equal to self.batch_size

        np.random.seed()
        si = np.random.choice(self.batch_size, n_styles)

        for l in range(self.y_dim):
            y = np.zeros(self.batch_size, dtype=np.int64) + l
            y_one_hot = np.zeros((self.batch_size, self.y_dim))
            y_one_hot[np.arange(self.batch_size), y] = 1

            samples = self.sess.run(self.fake_images,
                                    feed_dict={
                                        self.z: z_sample,
                                        self.y: y_one_hot
                                    })
            save_images(
                samples[:image_frame_dim * image_frame_dim, :, :, :],
                [image_frame_dim, image_frame_dim],
                check_folder(self.result_dir + '/' + self.model_dir) + '/' +
                self.model_name + '_epoch%03d' % epoch +
                '_test_class_%d.png' % l)

            samples = samples[si, :, :, :]

            if l == 0:
                all_samples = samples
            else:
                all_samples = np.concatenate((all_samples, samples), axis=0)
        """ save merged images to check style-consistency """
        canvas = np.zeros_like(all_samples)
        for s in range(n_styles):
            for c in range(self.y_dim):
                canvas[s * self.y_dim +
                       c, :, :, :] = all_samples[c * n_styles + s, :, :, :]

        save_images(
            canvas, [n_styles, self.y_dim],
            check_folder(self.result_dir + '/' + self.model_dir) + '/' +
            self.model_name + '_epoch%03d' % epoch +
            '_test_all_classes_style_by_style.png')

    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(self.model_name, self.dataset_name,
                                    self.batch_size, self.z_dim)

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir,
                                      self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir,
                                     self.model_name + '.model'),
                        global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir,
                                      self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess,
                               os.path.join(checkpoint_dir, ckpt_name))
            counter = int(
                next(re.finditer("(\d+)(?!.*\d)", ckpt_name)).group(0))
            print(" [*] Success to read {}".format(ckpt_name))
            return True, counter
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0


Using TensorFlow backend.


## gan.utils

In [0]:
"""
    Most codes from https://github.com/carpedm20/DCGAN-tensorflow
"""
from __future__ import division

import scipy.misc
import numpy as np

from six.moves import xrange
import matplotlib.pyplot as plt
import os, gzip

import tensorflow as tf
import tensorflow.contrib.slim as slim

from keras.datasets import cifar10
from keras.datasets import mnist


def one_hot(x, n):
    """
        convert index representation to one-hot representation
    """
    x = np.array(x)
    assert x.ndim == 1
    return np.eye(n)[x]


def prepare_input(data=None, labels=None):
    image_height = 32
    image_width = 32
    image_depth = 3
    assert (data.shape[1] == image_height * image_width * image_depth)
    assert (data.shape[0] == labels.shape[0])
    # do mean normalization across all samples
    mu = np.mean(data, axis=0)
    mu = mu.reshape(1, -1)
    sigma = np.std(data, axis=0)
    sigma = sigma.reshape(1, -1)
    data = data - mu
    data = data / sigma
    is_nan = np.isnan(data)
    is_inf = np.isinf(data)
    if np.any(is_nan) or np.any(is_inf):
        print('data is not well-formed : is_nan {n}, is_inf: {i}'.format(
            n=np.any(is_nan), i=np.any(is_inf)))
    # data is transformed from (no_of_samples, 3072) to (no_of_samples , image_height, image_width, image_depth)
    # make sure the type of the data is no.float32
    data = data.reshape([-1, image_depth, image_height, image_width])
    data = data.transpose([0, 2, 3, 1])
    data = data.astype(np.float32)
    return data, labels


def read_cifar10(filename):  # queue one element
    class CIFAR10Record(object):
        pass

    result = CIFAR10Record()

    label_bytes = 1  # 2 for CIFAR-100
    result.height = 32
    result.width = 32
    result.depth = 3

    data = np.load(filename, encoding='latin1')

    value = np.asarray(data['data']).astype(np.float32)
    labels = np.asarray(data['labels']).astype(np.int32)

    return prepare_input(value, labels)


def load_cifar10(train):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    if (train == True):

        dataX = x_train.reshape([-1, 32, 32, 3])
        dataY = y_train

    else:
        dataX = x_test.reshape([-1, 32, 32, 3])
        dataY = y_test

    seed = 547
    np.random.seed(seed)
    np.random.shuffle(dataX)
    np.random.seed(seed)
    np.random.shuffle(dataY)

    y_vec = np.zeros((len(dataY), 10), dtype=np.float)
    for i, label in enumerate(dataY):
        y_vec[i, dataY[i]] = 1.0

    return dataX / 255., y_vec


def load_mnist(train = True):

    def extract_data(filename, num_data, head_size, data_size):
        with gzip.open(filename) as bytestream:
            bytestream.read(head_size)
            buf = bytestream.read(data_size * num_data)
            data = np.frombuffer(buf, dtype=np.uint8).astype(np.float)
        return data

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train = x_train.reshape((60000, 28, 28, 1))
    y_train = y_train.reshape((60000))

    x_test = x_test.reshape((10000, 28, 28, 1))
    y_test = y_test.reshape((10000))

    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

    if (train == True):
        seed = 547
        np.random.seed(seed)
        np.random.shuffle(x_train)
        np.random.seed(seed)
        np.random.shuffle(y_train)

        y_vec = np.zeros((len(y_train), 10), dtype=np.float)
        for i, label in enumerate(y_train):
            y_vec[i, y_train[i]] = 1.0
        return x_train / 255., y_vec
    
    else:
        seed = 547
        np.random.seed(seed)
        np.random.shuffle(x_test)
        np.random.seed(seed)
        np.random.shuffle(y_test)

        y_vec = np.zeros((len(y_test), 10), dtype=np.float)
        for i, label in enumerate(y_test):
            y_vec[i, y_test[i]] = 1.0
        return x_test / 255., y_vec
      


def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir


def show_all_variables():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)


def get_image(image_path,
              input_height,
              input_width,
              resize_height=64,
              resize_width=64,
              crop=True,
              grayscale=False):
    image = imread(image_path, grayscale)
    return transform(image, input_height, input_width, resize_height,
                     resize_width, crop)


def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)


def imread(path, grayscale=False):
    if (grayscale):
        return scipy.misc.imread(path, flatten=True).astype(np.float)
    else:
        return scipy.misc.imread(path).astype(np.float)


def merge_images(images, size):
    return inverse_transform(images)


def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    if (images.shape[3] in (3, 4)):
        c = images.shape[3]
        img = np.zeros((h * size[0], w * size[1], c))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w, :] = image
        return img
    elif images.shape[3] == 1:
        img = np.zeros((h * size[0], w * size[1]))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w] = image[:, :, 0]
        return img
    else:
        raise ValueError('in merge(images,size) images parameter '
                         'must have dimensions: HxW or HxWx3 or HxWx4')


def imsave(images, size, path):
    image = np.squeeze(merge(images, size))
    return scipy.misc.imsave(path, image)


def center_crop(x, crop_h, crop_w, resize_h=64, resize_w=64):
    if crop_w is None:
        crop_w = crop_h
    h, w = x.shape[:2]
    j = int(round((h - crop_h) / 2.))
    i = int(round((w - crop_w) / 2.))
    return scipy.misc.imresize(x[j:j + crop_h, i:i + crop_w],
                               [resize_h, resize_w])


def transform(image,
              input_height,
              input_width,
              resize_height=64,
              resize_width=64,
              crop=True):
    if crop:
        cropped_image = center_crop(image, input_height, input_width,
                                    resize_height, resize_width)
    else:
        cropped_image = scipy.misc.imresize(image,
                                            [resize_height, resize_width])
    return np.array(cropped_image) / 127.5 - 1.


def inverse_transform(images):
    return (images + 1.) / 2.


""" Drawing Tools """


# borrowed from https://github.com/ykwon0407/variational_autoencoder/blob/master/variational_bayes.ipynb
def save_scattered_image(z,
                         id,
                         z_range_x,
                         z_range_y,
                         name='scattered_image.jpg'):
    N = 10
    plt.figure(figsize=(8, 6))
    plt.scatter(z[:, 0],
                z[:, 1],
                c=np.argmax(id, 1),
                marker='o',
                edgecolor='none',
                cmap=discrete_cmap(N, 'jet'))
    plt.colorbar(ticks=range(N))
    axes = plt.gca()
    axes.set_xlim([-z_range_x, z_range_x])
    axes.set_ylim([-z_range_y, z_range_y])
    plt.grid(True)
    plt.savefig(name)


# borrowed from https://gist.github.com/jakevdp/91077b0cae40f8f8244a
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)


def save_matplot_img(images, size, image_path):
    # revice image data // M*N*3 // RGB float32 : value must set between 0. with 1.
    for idx in range(64):
        vMin = np.amin(images[idx])
        vMax = np.amax(images[idx])
        img_arr = images[idx].reshape(32 * 32 * 3, 1)  # flatten
        for i, v in enumerate(img_arr):
            img_arr[i] = (v - vMin) / (vMax - vMin)
        img_arr = img_arr.reshape(32, 32, 3)  # M*N*3

        plt.subplot(8, 8, idx + 1), plt.imshow(img_arr,
                                               interpolation='nearest')
    plt.axis("off")


    plt.savefig(image_path)


## Main

In [0]:
import tensorflow as tf
import os

base_dir = "./"

out_dir = base_dir + "mnist_clip1_sigma0.6_lr0.55"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

gpu_options = tf.GPUOptions(visible_device_list="0")
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                      gpu_options=gpu_options)) as sess:

    epoch = 100
    cgan = OUR_DP_CGAN(sess,
                       epoch=epoch,
                       batch_size=64,
                       z_dim=100,
                       epsilon=9.6,
                       delta=1e-5,
                       sigma=0.6,
                       clip_value=1,
                       lr=0.055,
                       dataset_name='mnist',
                       checkpoint_dir=out_dir + "/checkpoint/",
                       result_dir=out_dir + "/results/",
                       log_dir=out_dir + "/logs/",
                       base_dir=base_dir)

    cgan.build_model()
    print(" [*] Building model finished!")

    show_all_variables()
    cgan.train()
    print(" [*] Training finished!")


11493376/11490434 [==============================] - 1s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
 [*] Building model finished!
---------
Variables: name (type shape) [size]
---------
discriminator/d_conv1/w:0 (float32_ref 4x4x11x64) [11264, bytes: 45056]
discriminator/d_conv1/biases:0 (float32_ref 64) [64, bytes: 256]
discriminator/d_conv2/w:0 (float32_ref 4x4x64x128) [131072, bytes: 524288]
discriminator/d_conv2/biases:0 (float32_ref 128) [128, bytes: 512]
discriminator/d_bn2/beta:0 (float32_ref 128) [128, bytes: 512]
discriminator/d_bn2/gamma:0 (float32_ref 128) [128, bytes: 512]
discriminator/d_fc3/Matrix:0 (float32_ref 6272x1024) [6422528, bytes: 25690112]
discriminator/d_fc3/bias:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_bn3/beta:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_bn3/gamma:0 (float32_ref 1024) [1024, bytes: 4096]
discriminator/d_fc4/Matrix:0 (float32_ref 10

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:207: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration : 198 Eps: 6.088221617339909
Iteration : 298 Eps: 6.316992921084355
Iteration : 398 Eps: 6.505090299808932
Iteration : 498 Eps: 6.6696494264479
Iteration : 598 Eps: 6.79218255319497
Iteration : 698 Eps: 6.914715679942041
Iteration : 798 Eps: 7.037248806689111
Iteration : 898 Eps: 7.153695366365903
Iteration : 61 Eps: 7.238063480353059
Iteration : 161 Eps: 7.322431594340215
Iteration : 261 Eps: 7.406799708327371
Iteration : 361 Eps: 7.491167822314528
Iteration : 461 Eps: 7.5755359363016845
Iteration : 561 Eps: 7.659904050288841
Iteration : 661 Eps: 7.744272164275998
Iteration : 761 Eps: 7.8100417154796835
Iteration : 861 Eps: 7.871156710676289
Iteration : 24 Eps: 7.932271705872896
Iteration : 124 Eps: 7.993386701069502
Iteration : 224 Eps: 8.054501696266108
Iteration : 324 Eps: 8.115616691462714
Iteration : 424 Eps: 8.176731686659322
Iteration : 524 Eps: 8.237846681855927
Iteration : 624 Eps: 8.298961677052533
Iteration : 724 Eps: 8.36007667224914
Iteration : 824 Eps: 8.421191